# Time series forecasting with sktime
Estimated time: 40 min

![](./img/tasks-forecasting.png)

## Agenda

- Quickstart
- Univariate forecasting
    - With statistical models
    - With machine learning models
    - Model evaluation and selection
- Univariate forecasting with exogenous data
- Multivariate forecasting
- Probabilistic forecasting
- Hierarchical forecasting

## Quickstart

* typical business use case :-)
* here's some monthly historic sales data

In [ ]:
from sktime.datasets import load_shampoo_sales
from sktime.utils.plotting import plot_series

y = load_shampoo_sales()

fig, ax = plot_series(y)

In [ ]:
from sktime.forecasting.model_selection import temporal_train_test_split

y_train, y_test = temporal_train_test_split(y=y, test_size=6)
fig, ax = plot_series(y_train, y_test, labels=["train", "test"])

In [ ]:
from sktime.forecasting.arima import AutoARIMA

# 1) Define the model
forecaster = AutoARIMA(suppress_warnings=True)

# 2) Fit on train data
forecaster.fit(y_train)

# 3) Use fitted model to predict for a certain forecast horizon (fh)
fh = [1, 2, 3, 4, 5, 6] # Relative to y_train
y_pred = forecaster.predict(fh)

fig, ax = plot_series(y_train, y_test, y_pred, labels=["train", "test", "pred"])

* scoring using `sktime` performance metrics
* requires forecasts & true values as `sktime` compatible time series

In [ ]:
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

smape = MeanAbsolutePercentageError(symmetric=True)

print(f"AutoARIMA - sMAPE error: {smape(y_test, y_pred):.1%}")

Notes:

* MAPE just for illustration - not always best choice
* for robust evaluation & comparison, use backtesting (not single train/test split)

## Univariate forecasting

showcase common approaches for forecasting univariate time series in `sktime`:
- Classical statistical models (e.g., econometric, ARIMA, etc)
- Machine learning models (e.g., direct/recursive reduction)

Recommendation: try simple models and naive baselines first!

### Classical forecasting method example: `AutoETS`

In [ ]:
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.theta import ThetaForecaster

forecasters = [AutoETS(auto=True), ThetaForecaster()]

for forecaster in forecasters:
    y_pred = forecaster.fit_predict(y=y_train, fh=fh)
    title = (
        f"{str(forecaster).split('(')[0]} - sMAPE error: {smape(y_test, y_pred):.1%}"
    )
    fig, ax = plot_series(
        y_train, y_test, y_pred, labels=["train", "test", "pred"], title=title
    )

Check out all other sktime forecasting algorithms [in the documentation](https://www.sktime.net/en/latest/api_reference/forecasting.html) or by running the code below:

In [ ]:
from sktime.registry import all_estimators

all_estimators("forecaster", as_dataframe=True, suppress_import_stdout=False)

### Forecasting with ML algorithms (reduction)

- uses sklearn regressor on tabulated data to forecast
- plug & play any sklearn compatible regressor, e.g., lightgbm or xgboost
- important: forecasting != regression

Estimator does this internally:

![](../images/tabularization.png)

in unified forecasting interface! No need to handle `sklearn` directly

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sktime.forecasting.compose import make_reduction

# Can be swapped with XBGoost, LightGBM, CatBoost, etc.
regressor = HistGradientBoostingRegressor()

# Create a forecaster from the tabular regressor by wrapping it in `make_reduction`
forecaster = make_reduction(regressor, strategy="direct", window_length=16)

y_pred = forecaster.fit_predict(y=y_train, fh=fh)
title = f"Gradient-boosted tree regressor - sMAPE error: {smape(y_test, y_pred):.1%}"
fig, ax = plot_series(
    y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"], title=title
)

... this is bad! Why?

Subtle:

- Gradient boosting trees cannot "extrapolate"
- only forecast well within their observed range

Solution: make (more) stationary by differencing

easy to do in `sktime`: transformers (= transformation estimators)

(note: wider concept than deep learn transformers, includes simple trafos too)

Let's see how to use the `Differencer` transformer:

In [ ]:
from sktime.transformations.series.difference import Differencer

transformer = Differencer(lags=1)
y_transform = transformer.fit_transform(y)
fig, ax = plot_series(
    y, y_transform, labels=["y", "y_diff"], title="Difference transformation"
)

Transformers composable with forecasters, plug together to forecaster!

here: plug `Differencer` into tree-based reduction forecaster, via `*` dunder:

In [ ]:
regressor = HistGradientBoostingRegressor()
forecaster = make_reduction(regressor, strategy="direct", window_length=16)
forecaster_with_differencer = Differencer(lags=1) * forecaster

y_pred = forecaster_with_differencer.fit_predict(y=y_train, fh=fh)
title = f"Gradient-boosted tree regressor with difference transform - sMAPE error: {smape(y_test, y_pred):.1%}"
fig, ax = plot_series(
    y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"], title=title
)

More on transformers and composition later!

### Model evaluation and selection

Best practice for model evaluation: backtesting, sliding window

(not single split MAPE etc...)

how this works:

* define backtesting schema using cross-validation splitter
* simple workflows: use `evaluate` all-in-one evaluator
* customizable: use benchmarking module for experiments

#### Window splitters

In [ ]:
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from sktime.utils.plotting import plot_windows

cv = ExpandingWindowSplitter(initial_window=24, fh=fh, step_length=2)
n_folds = cv.get_n_splits(y)
plot_windows(cv, y)

#### Backtesting for single model evaluation

We can leverage this more rebust cross-validation strategies for sinlge model evaluation by using `sktime`'s evaluat function.

In [ ]:
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError

forecaster = forecaster_with_differencer.clone()
scorers = [smape, MeanSquaredError(square_root=True)]
backtest = evaluate(
    forecaster=forecaster, y=y, cv=cv, scoring=scorers, return_data=True
)
backtest

In [ ]:
backtest.loc[0, "y_pred"]

In [ ]:
fig, ax = plot_series(
    y,
    *tuple(backtest["y_pred"][i] for i in range(n_folds)),
    labels=["y"] + [f"fold_{i}" for i in range(n_folds)],
    title=f"Backtest predictions, average sMAPE: {backtest['test_MeanAbsolutePercentageError'].mean():.1%}",
)

#### Advanced benchmarking via `ForecastingBenchmark`

* group of models in same benchmark scenario
* flexible set-up, e.g., splitters, evaluation metrics

Note: requires soft dependency [`kotsu`](https://github.com/datavaluepeople/kotsu) installed.

Start: create `ForecastingBenchmark` object

In [ ]:
from sktime.benchmarking.forecasting import ForecastingBenchmark

benchmark = ForecastingBenchmark()

step 2: add models with `add_estimator` method.

Example: `NaiveForecaster` as simple benchmark model.

Good idea to assess baseline prediction accuracy!

In [ ]:
from sktime.forecasting.naive import NaiveForecaster

benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="mean", window_length=3),
    estimator_id="Naive-mean-3-v1",
)
benchmark.add_estimator(estimator=AutoARIMA(), estimator_id="AutoARIMA-v1")
benchmark.add_estimator(estimator=AutoETS(auto=True), estimator_id="AutoETS-v1")
benchmark.add_estimator(
    estimator=forecaster_with_differencer.clone(), estimator_id="LightGBM-v1"
)

Next - define backtest strategy `cv`, define forecasting tasks & scoring:

In [ ]:
cv = ExpandingWindowSplitter(initial_window=24, fh=fh, step_length=2)
scorers = [smape]

benchmark.add_task(
    load_shampoo_sales,
    cv,
    scorers,
)

`run` to start benchmarking, this will now compute results:

In [ ]:
results_df = benchmark.run(output_file="results.csv")
results_df.set_index("model_id").iloc[:, -2:].style.format("{:.1%}")

Currently in active development! (summer programme)

- Open to feedback
- Welcome and appreciate contributions

## Univariate forecasting with exogenous data

- exogeneous data = other related time series that can improve prediction
- Example: information about promotions when forecasting sales (promotions drive sales)


We start by loading the same sales data we have been working on before.

In [ ]:
from sktime.datasets import load_shampoo_sales

y = load_shampoo_sales()

Let's use the sales data, noise and some simple transformations to create *fake* promotion

In [ ]:
import numpy as np
from sktime.utils.plotting import plot_series
from sktime.transformations.series.difference import Differencer

# Use a differencer, clipping and some noise to generate fake promotional data
transformer = Differencer(lags=1)
y_transform = transformer.fit_transform(y)
noise = np.random.RandomState(seed=93).normal(0, 100, np.shape(y))
X_promo = (y_transform + noise).clip(lower=0)

fig, ax = plot_series(
    y, X_promo, labels=["y", "fake_promotions"], title="Sales and Promotions"
)

We can split both the target time series (y: sales) and the exogenous time series (X: promotions) with the `temporal_train_test_split` we have used before.

In [ ]:
from sktime.forecasting.model_selection import temporal_train_test_split

fh = [1, 2, 3, 4]
y_train, y_test, X_train, X_test = temporal_train_test_split(
    y=y, X=X_promo, test_size=len(fh)
)

fig, ax = plot_series(
    y_train,
    X_train,
    y_test,
    X_test,
    labels=["y_train", "X_train", "y_train", "X_test"],
    title="Test and train: sales and promotions",
)

Now we can forecast y (sales) also using the known values of future X (promotions) by passing the future X data in the predict step.

In [ ]:
from sktime.forecasting.arima import AutoARIMA

forecaster = AutoARIMA(suppress_warnings=True)

# Use train data in fit
forecaster.fit(y=y_train, X=X_train, fh=fh)

# Note how the "future" data of X is passed in the predict step
y_pred = forecaster.predict(X=X_test)

Let's see how the prediction looks like when adding promotional data.

In [ ]:
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

smape = MeanAbsolutePercentageError(symmetric=True)

title = f"AutoARIMA with promotional data - sMAPE error: {smape(y_test, y_pred):.1%}"
fig, ax = plot_series(
    y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"], title=title
)

Note: as we created the promotions from the sales data, the performance upflift is over-optimistic (data leakage).

But what if don't have future promotional data?

If we believe that we can forecast `X` (promotions) independently of `y` (sales) we can use these predictions of `X` to inform the predictions of `y`.

Here, we decide to use a different model for `X` than for `y`:
- y (sales): AutoARIMA
- X (promotion): Croston - due to intermittency


In [ ]:
from sktime.forecasting.compose import ForecastX
from sktime.forecasting.croston import Croston

forecaster_X = ForecastX(
    forecaster_y=AutoARIMA(suppress_warnings=True),
    forecaster_X=Croston(),
)
forecaster_X.fit(y=y, X=X_promo, fh=fh)

After fitting on both `X` and `y` we can creat predictions of `y` directly. Under the hood `sktime` is forecasting `X` with the `Croston()` model and using it in the prediction step of `y`.

In [ ]:
# Now in the `predict` step we don't need to pass X
y_pred = forecaster_X.predict(fh=fh)

title = f"ForecastX: Using AutoARIMA for y (sales) and Croston for X (promotions)"
fig, ax = plot_series(
    y, y_pred, labels=["y", "y_pred"], title=title
)

- Sometimes, the focus is not on a single univariate time series, but rather on forecasting a group of time series that represent different aspects of a single entity.
- Example: forecasting multiple macroeconomic indicators that collectively measure "the economy".

Let's explore how `sktime` enables multivariate forecasting for this use-case.

## Multivariate forecasting

dataset: multiple macroecon indicators, reported yearly ("Longley dataset")

In [ ]:
from sktime.datasets import load_longley

_, y = load_longley()

y = y.drop(columns=["UNEMP", "ARMED", "POP"])

y

some forecasters, e.g., `VAR` are genuinely mutltivariate.

Let's predict:

In [ ]:
from sktime.forecasting.var import VAR

forecaster = VAR()
forecaster.fit(y, fh=[1, 2, 3])

y_pred = forecaster.predict()
y_pred

multivariate/univariate is visible and searchable via tags,

e.g., `get_tags` to display properties of forecaster:

In [ ]:
forecaster.get_tags()

But also all univariate forecasters can natively forecast multivariate data!

This is done by "broadcasting" across variables.

`ARIMA` is a purely univariate model:

In [ ]:
from sktime.datasets import load_longley
from sktime.forecasting.arima import ARIMA

_, y = load_longley()

y = y.drop(columns=["UNEMP", "ARMED", "POP"])

forecaster = ARIMA()
forecaster.fit(y, fh=[1, 2, 3])

forecaster.forecasters_

`sktime` fits one single `ARIMA` model per variable.

Tags tell us that `ARIMA` is univariate, using `get_tags`:

In [ ]:
forecaster.get_tags()

## Probabilistic forecasting

- Point predictions are often not enough!
- forecasts inherently contain some level of uncertainty
- important to estimate that uncertainty, "probabilistic predictions"
- example: uncertainty range from prediction intervals

`sktime` can make multiple types of probabilistic predictions

1st example: prediction intervals via `predict_interval`

In [ ]:
from sktime.datasets import load_shampoo_sales

y = load_shampoo_sales()

In [ ]:
from sktime.forecasting.ets import AutoETS

# 1) Define the model
forecaster = AutoETS(auto=True)

# 2) Fit on train data
forecaster.fit(y_train)

# 3) Use fitted model to predict for a certain forecast horizon (fh)
fh = [1, 2, 3, 4]
y_pred = forecaster.predict(fh)

# 4) Call a probabilistic method after or in place of step 3
y_pred_int = forecaster.predict_interval(coverage=0.95)

fig, ax = plot_series(
    y_train, y_test, y_pred, labels=["train", "test", "pred"], pred_interval=y_pred_int
)

In [ ]:
y_pred_int

Methods available for all probabilistic forecasters:

- `predict_interval` produces interval forecasts.
  Argument `coverage` (nominal interval coverage) must be provided.
- `predict_quantiles` produces quantile forecasts.
  Argument `alpha` (quantile values) must be provided.
- `predict_var` produces variance forecasts. Same args as `predict`.
- `predict_proba` produces full distributional forecasts. Same args as `predict`.

| Name | param | prediction/estimate of | `sktime` |
| ---- | ----- | ---------------------- | -------- |
| point forecast | | conditional expectation $\mathbb{E}[y'\|y]$ | `predict` |
| variance forecast | | conditional variance $Var[y'\|y]$ | `predict_var` |
| quantile forecast | $\alpha\in (0,1)$ | $\alpha$-quantile of $y'\|y$ | `predict_quantiles` |
| interval forecast | $c\in (0,1)$| $[a,b]$ s.t. $P(a\le y' \le b\| y) = c$ | `predict_interval` |
| distribution forecast | | the law/distribution of $y'\|y$ | `predict_proba` |

`all_estimators` use to list all forecasters capable of proba predictions:

In [ ]:
from sktime.registry import all_estimators

all_estimators(
    "forecaster",
    filter_tags={"capability:pred_int": True},
    as_dataframe=True,
    suppress_import_stdout=False,
)

Note:  estimators with`pred_int` tag always all probabilistic methods available - either all of them or none.

Presenting different proba methods and their outputs:

In [ ]:
forecaster.predict_interval(coverage=0.95)

In [ ]:
forecaster.predict_quantiles(alpha=[0.275, 0.95])

In [ ]:
forecaster.predict_var()

To predict full predictive distributions, `predict_proba` can be used. This returns a `BaseDistribution` child instance.

In [ ]:
forecaster.predict_proba()

## Hierarchical forecasting

![](../images/hierarchy.png)

Consider hierarchical dataframe of historical monthly sales in different categories:

In [ ]:
from odsc_utils import load_product_hierarchy

y = load_product_hierarchy()

y

We can pick a specific date to clearly see the hierarchy.

In [ ]:
# Multiindex slicing can become important when using hierarchical data!
y.loc[(slice(None), slice(None), "2000-01")]

Visualizing the different time series in the hierarchy:

In [ ]:
product_index = y.droplevel(-1).index.unique()
fig, ax = plot_series(*(y.loc[idx] for idx in product_index), labels=product_index, title="Product sales")

`sktime` broadcasts all non-hierarchical forecasters to hierarchical data:

In [ ]:
from sktime.forecasting.ets import AutoETS

forecaster = AutoETS(auto=True)

y_pred = forecaster.fit_predict(y, fh=[1])
y_pred

In [ ]:
forecaster.forecasters_

For hierarchies, often want forecasts of aggregated levels.

Instead of manually summing up,

we can use `Aggregator` transformer in `sktime`:

In [ ]:
from sktime.transformations.hierarchical.aggregate import Aggregator

y_hier = Aggregator().fit_transform(y)

y_hier.loc[(slice(None), slice(None), "2000-01")]

In [ ]:
forecaster = AutoETS(auto=True, random_state=0)

y_hier_pred = forecaster.fit_predict(y_hier, fh=1)
y_hier_pred

Compare top level and the sum of the bottom level forecasts...

... they do not add up!

In [ ]:
584.481241 - (119.460419 + 169.749332 + 146.466778 + 139.583316)

Why? Independent instances of forecaster fitted per level,

no constraint to ensure the predictions add up.

In [ ]:
forecaster.forecasters_

Use `ReconcilerForecaster` to enfore hierarchical reconciliation (= level forecasts add up)

`ReconcilerForecaster` takes a forecaster and adds reconciliation logic:

In [ ]:
from sktime.forecasting.reconcile import ReconcilerForecaster

reconciler_forecaster = ReconcilerForecaster(
    forecaster=forecaster.clone(), method="bu"
)

y_hier_pred = reconciler_forecaster.fit_predict(y_hier, fh=1)
y_hier_pred

Now top/bottom forecasts add up!

In [ ]:
575.259845 - (119.460419 + 169.749332 + 146.466778 + 139.583316)

Available reconciliation in docstring or `METHOD_LIST`:

In [ ]:
print("Valid reconciliation methods:")
for method in ReconcilerForecaster.METHOD_LIST:
    print(f"- {method}")

`HierarchyEnsembleForecaster` to customize forecasters at different hierarchy levels/nodes.

Forecasters built this way also aggregate the hierachical data for you under the hood.

In [ ]:
from odsc_utils import load_product_hierarchy

y = load_product_hierarchy()

In [ ]:
from sktime.forecasting.compose import HierarchyEnsembleForecaster
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.ets import AutoETS

forecasters = [
    ('Auto ARIMA', AutoARIMA(), 0),
    ('Auto ETS', AutoETS(auto=True), 1)
]

forecaster = HierarchyEnsembleForecaster(
                forecasters=forecasters,
                by='level', default = AutoETS(auto=True)
)

y_pred = forecaster.fit_predict(y, fh=[1])

y_pred

Up to now, all forecasts "per group"...

- Local: fit a model to each time series locally
- Global: fit a single model to all the series - below

### Global forecasting

Benefits of global forecasting:
- The model has access to more data to learn from, great when individual time series are short.
- Faster than local approach
- Empirically shown to outperform local models (e.g. M5 forecasting competition)

Note: global models assume the data generating procress for the group of time series is the same or at least similar.

In [ ]:
from odsc_utils import load_product_hierarchy
from sktime.forecasting.model_selection import temporal_train_test_split

y = load_product_hierarchy()

y_train, y_test = temporal_train_test_split(y_hier, test_size=4)

y_test

Let's begin by using local forecasting with the gradient boosting regressor we used previously to forecast the hierarchical data.

In [ ]:
regressor = HistGradientBoostingRegressor()
forecaster = make_reduction(regressor, strategy="direct", window_length=12, pooling="local")

y_pred = forecaster.fit_predict(y_train, fh=[1, 2, 3, 4])

In [ ]:
forecaster.forecasters_

We can adapt the error metrics to a hierarchical setting by using `multilevel` argument to obtain scores for each level.

In [ ]:
hier_smape = MeanAbsolutePercentageError(symmetric=True, multilevel="raw_values")
errors_local = hier_smape(y_test, y_pred)
errors_local

Now with the same regressor, we do global forecasting by setting the `pooling` argument to `global`.

In [ ]:
regressor = HistGradientBoostingRegressor()
forecaster = make_reduction(regressor, strategy="direct", window_length=12, pooling="global")

y_pred = forecaster.fit_predict(y_train, fh=[1, 2, 3, 4])

Let's compare the scores for the `local` and `global` approach.

In [ ]:
errors_global = hier_smape(y_test, y_pred)

print(f"Average sMAPE with local pooling: {errors_local.mean().iloc[0]:.1%}")
print(f"Average sMAPE with global pooling: {errors_global.mean().iloc[0]:.1%}")

Quick recap of what we have covered in this notebook:

- Univariate forecasting (stats and ML)
- Univariate with exogenous data
- Multivariate forecasting
- Probabilistic forecasting
- Hierarchical forecasting

### Credits: notebook 3 - forecasting

notebook creation: marrov, fkiraly

partly based on:

* pydata 2022 Berlin notebooks (fkiraly, danbartl)
* sktime forecasting tutorial (fkiraly, mloning and others)

sktime forecasting module: [many contributors](https://www.sktime.net/en/latest/about/contributors.html)